# Homework 2

## FINM 37400 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

***

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (A): US Treasuries in November 2008

## Data
* Use the data file `treasury_ts_2015-08-15.xlsx`.
* Examine the treasure issues with `kytreasno` of `204046` and `204047`. These are the bond and note (respectively) which mature on 2015-08-15.
* Look at the data on 2008-11-04.

## 1.1 The situation

Make a chart comparing the issues in the following features, (as of Nov 4, 2008.)
* coupon rate
* bid
* ask
* accrued interest
* dirty price
* duration (quoted in years, not days, assuming 365.25 days per year.)
* modified duration
* YTM

## 1.2 Hedge Ratio

Suppose you are building a trade to go long $n_i$ bonds (`204046`) and short $n_j$ notes (`204047`).

We can find an equation for $n_j$ in terms of $n_i$ such that the total holdings will have duration equal to zero. (Having zero duration also means have zero dollar duration, if helpful.)

Notation:
* $n_i$: number of bonds purchased (or sold)
* $D_i$: duration of bond $i$
* $D_{\$,i}$: dollar duration of bond $i$, equal to $p_iD_i$

If we want the total duration of our holdings to be zero, then we need to size the trade such that $n_i$ and $n_j$ satisfy,

$$0 = n_iD_{\$,i} + n_jD_{\$,j}$$

$$n_j = -n_i\frac{D_{\$,i}}{D_{\$,j}}$$

Suppose you will use \\$1mm of capital, leveraged 50x to buy \\$50mm of the bonds (`204046`).

Use the ratio above to short a number of notes (`204047`) to keep zero duration.

Report the number of bonds and notes of your position, along with the total dollars in the short position.

In [69]:
import pandas as pd
import numpy as np
import datetime
import warnings
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize
%matplotlib inline
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
from treasury_cmds import *

In [2]:
treasury_path = 'C:/Users/dcste/OneDrive/fixed_income/fixed_income_FORKED/finm-fixedincome-2023/data/treasury_ts_2015-08-15.xlsx'
t_path_2 = 'C:/Users/dcste/OneDrive/fixed_income/fixed_income_FORKED/finm-fixedincome-2023/data/treasury_quotes_2022-09-30.xlsx'
info = pd.read_excel(treasury_path, sheet_name='info', index_col = 0)
ts_data = pd.read_excel(treasury_path, sheet_name= 'database')
ts_data.columns = ts_data.columns.str.upper()
treasury_df = pd.read_excel(t_path_2)
treasury_df.columns = treasury_df.columns.str.upper()
treasury_df.sort_values('TMATDT', inplace=True)
treasury_df.set_index("KYTREASNO", inplace = True)


In [3]:
mask = ts_data[ts_data.CALDT == '2008-11-04']
metrics = mask.copy()[['KYTREASNO','CALDT','TDBID','TDASK','TDNOMPRC','TDACCINT', 'TDYLD','TDDURATN','TDPUBOUT']].set_index('KYTREASNO')
metrics.columns = ['CALDT','BID','ASK', 'Nominal_Price','Accrued Interest','YTM','Duration','Outstanding']

In [4]:
metrics['Dirty_Price'] = (metrics['BID']+metrics['ASK'])*.5 + metrics['Accrued Interest']

In [5]:
metrics['YTM'] *= 365.25
metrics['Coupon_Rate'] = [4.25,10.625]
metrics['Duration'] /= 365.25
metrics['Modified Duration'] = metrics['Duration']/(1+(metrics['YTM']/2))
metrics['Dollar_Duration'] = metrics['Modified Duration']*metrics['Dirty_Price']
metrics = metrics.T

In [6]:
metrics

KYTREASNO,204047,204046
CALDT,2008-11-04 00:00:00,2008-11-04 00:00:00
BID,105.953125,141.859375
ASK,105.984375,141.890625
Nominal_Price,105.96875,141.875
Accrued Interest,0.935462,2.338655
YTM,0.032362,0.035753
Duration,5.935706,5.230138
Outstanding,20998.0,2852.0
Dirty_Price,106.904212,144.213655
Coupon_Rate,4.25,10.625


In [100]:
trade_pair = pd.DataFrame(data = None, columns=['204047','204046'])
trade_pair.loc['YTM'] = [0.032362,0.035753]
trade_pair.loc['Dirty_Price']  = [106.904212,144.213655]

trade_pair.loc['Modified_Duration'] = [5.841189,5.13824]
trade_pair.loc['dollar_duration'] = [624.447664,741.010749]

In [101]:
trade_pair

,204047,204046
YTM,0.03,0.04
Dirty_Price,106.90,144.21
Modified_Duration,5.84,5.14
dollar_duration,624.45,741.01


In [102]:
long_security = trade_pair.loc['YTM'].idxmax()
short_security = trade_pair.loc['YTM'].idxmin()

short_security

'204047'

In [63]:
p = trade_pair.loc['Dirty_Price']
p.loc['204046']

144.213655

# Calculating Hedge Ratio

## Financing Assumptions

- In this trade we take a long position in the bond which has the higher yield and short the note that has a lower yield.

In [80]:
def hedge_ratio(dollar_duration_long, dollar_duration_short, size_long = None):
    if size_long == None:
        return -dollar_duration_long/dollar_duration_short
    else:
        return (-size_long*(dollar_duration_long))/dollar_duration_short

In [92]:
pd.options.display.float_format = '{:,.2f}'.format

In [105]:
LONG_EQUITY = 1e6
def trade_balance_sheet(prices, durations, haircuts, long_equity, short_equity):
    balance_sheet = pd.DataFrame(data = None,columns=['Equity','Asset_Value','Dollar_Duration','Contracts'], index = [long_equity,short_equity])
    long_price = prices.loc[long_equity]
    short_price = prices.loc[short_equity]
    duration_long = durations.loc[long_equity]
    duration_short = durations.loc[short_equity]
    balance_sheet.loc[long_equity, 'Dollar_Duation'] = durations.loc[long_equity]
    balance_sheet.loc[short_equity,'Dollar_Duration'] = durations.loc[short_equity]
    haircut_long = haircuts.loc['long']
    haircut_short = haircuts.loc['short']
    balance_sheet.loc[long_equity,'Equity'] = LONG_EQUITY
    balance_sheet.loc[long_equity,'Asset_Value'] = 5e7
    balance_sheet.loc[long_equity,'Contracts'] = balance_sheet.loc[long_equity,'Asset_Value']/long_price
    balance_sheet.loc[short_equity,'Contracts'] = hedge_ratio(dollar_duration_long=duration_long,dollar_duration_short=duration_short,\
        size_long=balance_sheet.loc[long_equity,'Contracts'])
    balance_sheet.loc[short_equity,'Asset_Value'] = balance_sheet.loc[short_equity,'Contracts']*short_price
    balance_sheet.loc[short_equity,'Equity'] = balance_sheet.loc[short_equity,'Asset_Value']*haircut_short
    
    return balance_sheet
    
    

In [98]:
financing = pd.DataFrame(dtype='float64',index=['long','short'])
financing['haircut'] = [.02,.02]
financing['repo'] = [.0015,.0010]
financing

,haircut,repo
long,0.02,0.00
short,0.02,0.00


In [106]:
trade_balance_sheet(prices=trade_pair.loc['Dirty_Price'], durations=trade_pair.loc['dollar_duration'],haircuts = financing['haircut'],\
   long_equity = long_security, short_equity = short_security )


,Equity,Asset_Value,Dollar_Duration,Contracts,Dollar_Duation
204046,"1,000,000.00","50,000,000.00",NaN,"346,707.81",741.01
204047,"-879,664.15","-43,983,207.73",624.45,"-411,426.33",NaN


## 1.3 Profit Opportunity

Using the concept of **modified duration**, how much profit or loss (PnL) would you expect to make for every basis point of convergence in the spread? Specifically, assume the convergence is symmetric: the bond's (`204046`) ytm goes down 0.5bp and the note (`204047`) ytm goes up 0.5bp.

Describe the PnL you would expect to achieve on your position should this happen. Specify the PnL of the long position, the short position, and the net total.

Suppose the spread in YTM between the two securities disappears, due to a symmetric move of roughly ~17bps in each security's YTM. What is the PnL? (This is just a linearly scaling of your prior answer for a 1bp convergence.) 


## 1.4 Result in 2008

Calculate the profit (or loss) on the position on the following two dates:
* 2008-11-25
* 2008-12-16

To calculate the pnl on each date, simply use the prices of the securities on those dates along with your position sizes, ($n_i, n_j$). No coupon is being paid in November or December, so all you need is the "dirty" price on these two dates.

Does the pnl make sense (approximately) given your results in 1.3 with regard to the sensitivity of pnl to moves in the YTM spread?

## 1.5 *Optional*: Examining the Trade through June 2009

Calculate the pnl of the trade for the following dates:
* 2009-01-27
* 2009-03-24
* 2009-06-16

Did the trade do well or poorly in the first six months of 2009?

Calculate the YTM spreads on these dates. Does the YTM spread correspond to pnl roughly as we would expect based on the calculation in 1.3?

In [ ]:
#1.3
def pnl_spread_trade()

***

# 2 Calculating Duration

Use the data file `../data/treasury_quotes_2022-09-30.xlsx`.

This data reports duration as `TDDURATN`. It quotes the duration in days, so I recommend dividing by 365 to get the duration in its usual format.

## 2.1 
Set up the cashflow matrix. 


## 2.2
Build a discount curve assuming that the spot rate is 2\% per year, continuously compounded.

Plot the discount curve and the associated spot curve out to 30 years maturity.

*Note, you do not need to properly extract a spot curve and associated discount factors; rather, you are simply assuming a flat term structure of spot rates at 2\% and using that to figure out discount rates.*

## 2.3
For each treasury issue, calculate the duration as the weighted average of the (discounted!) cashflow maturity.

Report the summary statistics of the durations. (Use `.describe()` from pandas.)


## 2.4
How close are your duration estimates to the imputed durations given in the data source, (column `TDDURATN`)?

Report the summary statistics of the imputed durations minus your calculated durations from above.

Why might they be different?

## 2.5
Continue using your assumed discount rates of 2\% to calculate the convexity of each issue.

Report the summary statistics of these convexity calculations.

## 2.6 (Optional)

Re-do the duration and convexity calculations using an extracted discount curve instead of a discount curve based on a constant (arbitrary) spot rate. 

# 3 Hedging Duration

Import `treasury_ts_issue_duration_(207392, 207391, 207457).xlsx`.

I suggest using code such as

* `tsdata = pd.read_excel(filepath_tsdata,sheet_name='ts')`

* `tsdata.columns = tsdata.columns.str.upper()`

* `px = tsdata.pivot_table(index='CALDT',columns='KYTREASNO',values='TDASK').dropna()`
* `duration = tsdata.pivot_table(index='CALDT',columns='KYTREASNO',values='TDDURATN').dropna()`

## 3.1 

Suppose you have a portfolio of 10,000 USD long in security `207391` on the last day of the sample.

If you want to manage interest rate exposure using duration, how large of a short position should you hold in `207392`?

(Duration is the column `TDDURATN` in the raw data.)

## 3.2

Step through the time-series, doing the following:

* Starting at the end of the first day, set the hedged position according to the relative given durations.
* Use the second day's price data to evaluate the net profit or loss of the hedged position.
* Reset the the hedged position using the end-of-second-day durations. Again fix the long position of security `207391` to be 10,000.
* Repeat throughout the timeseries.

Report 
* the total profit (or loss.)
* the mean, standard deviation, min, and max of the daily profit or loss.

***

# 4 Other Interest-Rate Risks

## *Optional*

***No need to submit this problem, but if we discuss it, then you are expected to know it.***

## 4.1 Other Yield Curve Movements

Use the yield curve time-series data in `'../data/yields_2022-11-30.xlsx` to calculate the time-series of the **level**, **slope**, and curvature** factors.

Calculate the yield-curve factors. For each point in time, calculate the following three factors:

$\begin{align}
x^{\text{level}}_t =& \frac{1}{N_{\text{yields}}}\sum_{i=1}^{N_{\text{yields}}} y^{(i)}_t\\
x^{\text{slope}}_t =& y^{(30)}_t - y^{(1)}_t\\
x^{\text{curvature}}_t =& -y^{(1)}_t + 2 y^{(10)}_t - y^{(30)}_t
\end{align}$

Report the mean and volatility of each factor.

Report the correlation matrix of the factors.

## 4.2 Factor Duration

Calculate the factor duration of the treasuries from `treasury_ts_issue_duration_(207392, 207391, 207457).xlsx`.

Run a multivariate regression of the bond prices on all three factors constructed above from the yield factors: level, slope, and curvature. 

Estimate the regression in the form of day-over-day differences for both bond prices and factors. That is, we are using regression to approximate the factor duration equation,

$\begin{align}
\frac{dP}{P} = \beta_1 dz_1 + \beta_2 dz_2 + \beta_3 dz_3 + \epsilon
\end{align}$

Report the betas for each of these factors, for each of the bond prices.